<a href="https://colab.research.google.com/github/dantae74/machine-learning/blob/main/12-01-rnn-hello.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

모두를 위한 머신러닝에서 가져왔습니다.
# RNN - 'hihello'

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
idx2char = ['h', 'i', 'e', 'l', 'o']
# Teach hello: hihell -> ihello
# x_data = [[0, 1, 0, 2, 3, 3]]  # hihell
y_data = [[1, 0, 2, 3, 3, 4]]  # ihello

num_classes = 5
input_dim = 5  # one-hot size, same as hidden_size to directly predict one-hot
sequence_length = 6  # |ihello| == 6
learning_rate = 0.1

x_one_hot = np.array([[[1, 0, 0, 0, 0],    # h 0
                       [0, 1, 0, 0, 0],    # i 1
                       [1, 0, 0, 0, 0],    # h 0
                       [0, 0, 1, 0, 0],    # e 2
                       [0, 0, 0, 1, 0],    # l 3
                       [0, 0, 0, 1, 0]]],  # l 3
                     dtype=np.float32)

y_one_hot = tf.keras.utils.to_categorical(y_data, num_classes=num_classes)
print(x_one_hot.shape)
print(y_one_hot.shape)

(1, 6, 5)
(1, 6, 5)


In [3]:
tf.model = tf.keras.Sequential()

# make cell and add it to RNN layer
# input_shape = (1,6,5) => number of sequence (batch), length of sequence, size of input dim
# cell = tf.keras.layers.LSTMCell(units=num_classes, input_shape=(sequence_length, input_dim))
# cell = tf.keras.layers.LSTMCell(units=num_classes)
# tf.model.add(tf.keras.layers.RNN(cell=cell, return_sequences=True))
tf.model.add(tf.keras.layers.LSTM(units=num_classes, return_sequences=True))

# single LSTM layer can be used as well instead of creating LSTMCell
# tf.model.add(tf.keras.layers.LSTM(units=num_classes, input_shape=(sequence_length, input_dim), return_sequences=True))

# fully connected layer
tf.model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units=num_classes, activation='softmax')))

tf.model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=learning_rate),
                 metrics=['accuracy'])

In [4]:
# train
tf.model.fit(x_one_hot, y_one_hot, epochs=50, verbose=0)
tf.model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 6, 5)              220       
_________________________________________________________________
time_distributed (TimeDistri (None, 6, 5)              30        
Total params: 250
Trainable params: 250
Non-trainable params: 0
_________________________________________________________________


In [5]:
predictions = tf.model.predict(x_one_hot)
for i, prediction in enumerate(predictions):
    print(prediction)
    # print char using argmax, dict
    result_str = [idx2char[c] for c in np.argmax(prediction, axis=1)]
    print("\tPrediction str: ", ''.join(result_str))

[[1.4605483e-03 9.9545622e-01 1.2130295e-03 1.8597755e-03 1.0409874e-05]
 [9.9614477e-01 5.1702915e-05 1.8787251e-03 1.0797977e-03 8.4507931e-04]
 [8.8519679e-04 1.0277849e-03 9.9111301e-01 6.9649694e-03 9.0870690e-06]
 [9.4981646e-05 4.8281243e-05 2.4473509e-03 9.9730432e-01 1.0510432e-04]
 [7.1928278e-04 1.1667537e-05 1.2177313e-03 9.8936021e-01 8.6911181e-03]
 [2.0545321e-04 3.1218607e-07 8.1139915e-06 7.0822984e-03 9.9270380e-01]]
	Prediction str:  ihello
